In [ ]:
%%bash
pip install mxnet keras keras-mxnet matplotlib opencv-python sklearn

# Preparation

In [1]:
import os; os.environ['KERAS_BACKEND'] = 'mxnet'
import mxnet as mx
import numpy as np
import keras as k
from matplotlib.pyplot import imshow
import cv2

%matplotlib inline

Using MXNet backend


In [2]:
SHAPE=(224, 224, 3)
CLASSES=4
BATCH_SIZE = 64

In [3]:
import helpers.labelinputgenerator
trainData = helpers.labelinputgenerator.LabelInputGenerator("train", SHAPE, CLASSES, batch_size=BATCH_SIZE)
testData = helpers.labelinputgenerator.LabelInputGenerator("test", SHAPE, CLASSES, shuffle=False, batch_size=BATCH_SIZE)

print("Number of batches in the training data: ", len(trainData))
print("Number of batches in the Validation data: ", len(testData))

Number of batches in the training data:  980
Number of batches in the Validation data:  108


# Test image

In [ ]:
for x, y in trainData:
    i = 62
    imshow(x[i])
    print(y[i])
    break

# Setup NN

In [ ]:
import models.simplemobilenet as m

NNmodel = m.create_model(input_shape=SHAPE, outputs=CLASSES)
NNmodel.summary()

In [ ]:
NUM_EPOCHS = 1
opt = k.optimizers.Adam()
NNmodel.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])

In [ ]:
history = NNmodel.fit_generator(
	trainData,
	validation_data=testData,
	epochs=NUM_EPOCHS)

# serialize model to JSON
model_json = NNmodel.to_json()
with open("result/classification_model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5

NNmodel.save_weights("result/classification_model_weights.h5")
NNmodel.save("result/classification_model.h5.chkpt")
print("Saved model to disk")

# Load model

In [9]:
from keras.models import load_model
NNmodel = load_model('result/classification_model.h5.chkpt')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 16)      432       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 16)      64        
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 16)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 16)      144       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 16)      64        
__________

In [ ]:
# DELETE ME
from keras.models import model_from_json

with open('result/classification_model.json', 'r') as json_file:
    loaded_model_json = json_file.read()

NNmodel = model_from_json(loaded_model_json)
# load weights into new model
NNmodel.load_weights("result/classification_model.h5")
# summarize model.
NNmodel.summary()

opt = k.optimizers.Adam()
NNmodel.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])

# Evaluate Result

In [10]:
valData = helpers.labelinputgenerator.LabelInputGenerator("val", SHAPE, CLASSES, shuffle=False)
evaluation = NNmodel.evaluate_generator(valData)

/Users/msei/workspaces/ubiquitous-adventure/venv/lib/python3.7/site-packages/mxnet/module/bucketing_module.py:426: UserWarning: Optimizer created manually outside Module but rescale_grad is not normalized to 1.0/batch_size/num_workers (1.0 vs. 0.03125). Is this intended?
  force_init=force_init)


In [12]:
print('Validation loss:', evaluation[0])
print('Validation accuracy:', evaluation[1])

Validation loss: 0.15657349732582984
Validation accuracy: 0.9481997282608695


In [11]:
import matplotlib.pyplot as plt
%matplotlib inline

accuracy = history.history['acc']
val_accuracy = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(accuracy))

plt.plot(epochs, accuracy, 'bo', label='Training accuracy')
plt.plot(epochs, val_accuracy, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

print('Validation loss:', evaluation[0])
print('Validation accuracy:', evaluation[1])

NameError: name 'history' is not defined

# Manual Evaluation

In [13]:
prediction = NNmodel.predict_generator(valData)
prediction_classes = np.argmax(prediction, axis=1)

__this is wrong, as we do multiclassification__

In [16]:
val_y = valData.all_labels()
val_y = val_y[:-1]

In [17]:
from sklearn.metrics import classification_report
target_names = ["None", "Cat", "Dog", "Human"]
print(classification_report(val_y, prediction_classes, target_names=target_names))

ValueError: Classification metrics can't handle a mix of multilabel-indicator and multiclass targets

In [22]:
i = 400
print(val_y[i])
print(prediction[i])

[0 0 0 1]
[1.0100494e-04 1.5787354e-02 4.5061439e-02 9.7743624e-01]
